<a href="https://colab.research.google.com/github/Patatone/Analysis-of-the-COVID-19-impact-on-LTE-Networks/blob/main/Analysis_of_the_COVID_19_impact_on_LTE_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
%pip install geopandas
%pip install kneed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:
import numpy as np
import pandas as pd
import geopandas
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import datetime
from sklearn.cluster import KMeans
from kneed import KneeLocator
from google.colab import files

# Import Data


In [136]:
from google.colab import drive
import glob
import os

# Use this Section to import the data files provided in the project folder.

# NETWORK KPI
# Location: Milan ; Reference month: either January, February or March 2020:

## Google drive required lines
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/MRN_data/'

## Local path required lines
# file_path = ''

# KPIs
# We select January, February and March
all_files = glob.glob(os.path.join(file_path , "Milano_800*.csv"))
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

data.drop(['Unnamed: 0'], axis=1, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# Cells Location:
locations = pd.read_csv(file_path+'Coordinates_MILANO.csv')

# Drop all possible duplicate rows and drop all possible rows with any NaN and NaT values
locations = locations.drop_duplicates().dropna()

# https://pandas.pydata.org

# This section shows some information regarding the dataset
print(20*'*')
print('Data types:\n')
print(data.dtypes)
print(20*'*')  
print('Number of data points: ', len(data))
print('Number of columns in the dataset: ', len(data.columns))
print(20*'*')
print(data.isnull().sum(axis=0)) # this command show the number of NON valid data points for each column of the dataset:
                                 # a KPI measure for some timestamp can get lost during the storing procedure
print(20*'*')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
********************
Data types:

REGIONE                    object
COMUNE                     object
Date               datetime64[ns]
ECELL_ID                   object
DL_BW                     float64
RRC_S_SR                  float64
RRC_RE_SR                 float64
ERAB_S_SR                 float64
CS_SR                     float64
IntraF_Hout_SR            float64
InterF_Hout_SR            float64
Hin_SR                    float64
DL_VOL                    float64
UL_VOL                    float64
DL_THR_AVG                float64
UL_THR_AVG                float64
DL_THR_MAX                float64
UL_THR_MAX                float64
USERNUM_MAX               float64
PRB_DL_Used_Avg           float64
PRB_UL_Used_Avg           float64
dtype: object
********************
Number of data points:  841085
Number of columns in the dataset:  21
*******************

In [137]:
# Here you can understand the size of the scenario, 
# i.e., how many cells you are considering.
print('Number of (distinct) cells: ', len(data.drop_duplicates(subset='ECELL_ID')))


Number of (distinct) cells:  398


# Data Pre-Processing

In [138]:
# We can split the dataset into tree perdios:
# Full lockdown -> from "9th March 2020" to "31th March 2020"(last data)
# Restrictions -> from "16th February 2020" to "9th March 2020"
# Covid free - > from "1 January 2020"(first data) to "16th February 2020"

full_lockdown_end_date = pd.Timestamp(year=2020, month=4, day=1, hour = 0, minute =1)
full_lockdown_start_date = pd.Timestamp(year=2020, month=3, day=8, hour = 23, minute =59)

restrictions_end_date = pd.Timestamp(year=2020, month=3, day=10, hour = 0, minute =1)
restrictions_start_date = pd.Timestamp(year=2020, month=2, day=15, hour = 23, minute =59)

covid_free_end_date = pd.Timestamp(year=2020, month=2, day=17, hour = 0, minute =1)
covid_free_start_date = pd.Timestamp(year=2019, month=12, day=12, hour = 23, minute =59)


full_lockdown = data[data['Date'] < full_lockdown_end_date]
full_lockdown = full_lockdown[full_lockdown['Date'] > full_lockdown_start_date]

restrictions = data[data['Date'] < restrictions_end_date]
restrictions = restrictions[restrictions['Date'] > restrictions_start_date]

covid_free = data[data['Date'] < covid_free_end_date]
covid_free = covid_free[covid_free['Date'] > covid_free_start_date]


In [139]:
# Typically, daily and night KPIs traces are analysed differently, as network users 
# show very different behaviours depending on the two moments. 
#In this section, the considered weekly data are grouped into Daily (from 6AM to 24 PM) 
# and Night (from 00 AM to 6 AM) Data

# January
#week_day = week.set_index('Date').between_time('06:00:00', '23:59:59')
#week_night = week.set_index('Date').between_time('00:00:00', '05:59:59')


In [140]:
def print_cluster_pattern(dataset):
  dt = dataset.copy()
  df1 = dt[dt['Clusters'] == 0]

  plt.plot(list(range(0,len(df1))), df1['DL_VOL'])
  plt.show()

  df2 = dt[dt['Clusters'] == 1]

  plt.plot(list(range(0,len(df2))), df2['DL_VOL'])
  plt.show()

  df3 = dt[dt['Clusters'] == 2]

  plt.plot(list(range(0,len(df3))), df3['DL_VOL'])
  plt.show()

In [ ]:
# CLUSTERING PART 

#filtering by weekdays and daily business hours
x = covid_free[covid_free['Date'].dt.hour.between(6, 24)]

hours = pd.to_datetime(x['Date']).dt.hour
weekdays = x['Date'].dt.weekday
x['Hour'] = hours
x['Weekday'] = weekdays


#creating dataset with relevant data from previous DataFrame
x = x[['ECELL_ID', 'DL_VOL', 'Hour', 'Weekday']]
#x = x.groupby(['ECELL_ID', 'Hour', 'Weekday']).mean()
x = x.reset_index()
#splitting between week and weekend days
x['Weekday'] = x['Weekday'].apply(lambda x : 0 if x < 5 else (1 if x == 5 else 2))
x = x.groupby(['ECELL_ID', 'Hour', 'Weekday']).mean()
x = x.reset_index()

kmeans_kwargs = {
    "init": "k-means++",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 0,
    }

# selecting the DL_Volume column as entry data for the k-means clustering algorithm
dl_array = x[['DL_VOL', 'Hour', 'Weekday']]
sse = []
for k in range(1, 11):
  # creating a 2 clusters k-means value clustering class and fitting the DataFrame accordingly
  kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
  identified_clusters = kmeans.fit(dl_array)
  sse.append(kmeans.inertia_)

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
print()

# Determining the elbow point in the SSE curve
kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
k = kl.elbow
print('Perfect "k" value for the KMeans algorithm:', k)
print()

# creating a "k" clusters k-means value clustering class and fitting the DataFrame accordingly
kmeans = KMeans(n_clusters=k).fit(dl_array)
identified_clusters = kmeans.fit_predict(dl_array)
data_with_clusters = x.copy()
data_with_clusters['Clusters'] = identified_clusters

# Scattering the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(data_with_clusters['Hour'], data_with_clusters['Weekday'], data_with_clusters['DL_VOL'], c = data_with_clusters['Clusters'],
            cmap='rainbow')
fig.colorbar(img)
plt.show()

print_cluster_pattern(data_with_clusters)

data_with_clusters = data_with_clusters.join(locations.set_index('ECELL_ID'), on='ECELL_ID')
data_with_clusters = data_with_clusters[['ENODEB_ID', 'Clusters']]

# We cannot use mean(), it's wrong. We have to use mode()
#data_with_clusters = data_with_clusters.groupby(['ECELL_ID']).mean().round()

# data_with_clusters.to_csv('df.csv')
# files.download('df.csv')

data_with_clusters = data_with_clusters.groupby('ENODEB_ID', as_index=False)['Clusters'].apply(lambda x: x.mode().iloc[0])
#data_with_clusters = data_with_clusters.groupby('ENODEB_ID')['Clusters'].apply(lambda x: x.value_counts().index[0]).reset_index()
print(len(data_with_clusters))

# Splitting residential cells from non-residential ones
support_residential = data_with_clusters[data_with_clusters['Clusters'] == 0]
print(len(support_residential))
support_promiscuous = data_with_clusters[data_with_clusters['Clusters'] == 1]
print(len(support_promiscuous))
support_business = data_with_clusters[data_with_clusters['Clusters'] == 2]
print(len(support_business))

support_residential = support_residential.reset_index()
support_business = support_business.reset_index()
support_promiscuous = support_promiscuous.reset_index()

data_enodebs = data.copy().join(locations.set_index('ECELL_ID'), on='ECELL_ID')
# Splitting all the data according to the clusters
residential_data = data.loc[data_enodebs['ENODEB_ID'].isin(support_residential['ENODEB_ID'])].reset_index(drop=True)
business_data = data.loc[data_enodebs['ENODEB_ID'].isin(support_business['ENODEB_ID'])].reset_index(drop=True)
promiscuous_data = data.loc[data_enodebs['ENODEB_ID'].isin(support_promiscuous['ENODEB_ID'])].reset_index(drop=True)

# Splitting all the locations according to the clusters
residential_locations = locations.loc[locations['ENODEB_ID'].isin(support_residential['ENODEB_ID'])].reset_index(drop=True)
business_locations = locations.loc[locations['ENODEB_ID'].isin(support_business['ENODEB_ID'])].reset_index(drop=True)
promiscuous_locations = locations.loc[locations['ENODEB_ID'].isin(support_promiscuous['ENODEB_ID'])].reset_index(drop=True)

display(residential_locations)
display(business_locations)
display(promiscuous_locations)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# We can split the CLUSTERED dataset into tree perdios:
# Full lockdown -> from "9th March 2020" to "31th March 2020"(last data)
# Restrictions -> from "16th February 2020" to "9th March 2020"
# Covid free - > from "1 January 2020"(first data) to "16th February 2020"

full_lockdown_residential = residential_data[residential_data['Date'] < full_lockdown_end_date]
full_lockdown_residential = full_lockdown_residential[full_lockdown_residential['Date'] > full_lockdown_start_date]

full_lockdown_business = business_data[business_data['Date'] < full_lockdown_end_date]
full_lockdown_business = full_lockdown_business[full_lockdown_business['Date'] > full_lockdown_start_date]

restrictions_residential = residential_data[residential_data['Date'] < restrictions_end_date]
restrictions_residential = restrictions_residential[restrictions_residential['Date'] > restrictions_start_date]

restrictions_business = business_data[business_data['Date'] < restrictions_end_date]
restrictions_business = restrictions_business[restrictions_business['Date'] > restrictions_start_date]

covid_free_residential = residential_data[residential_data['Date'] < covid_free_end_date]
covid_free_residential = covid_free_residential[covid_free_residential['Date'] > covid_free_start_date]

covid_free_business = business_data[business_data['Date'] < covid_free_end_date]
covid_free_business = covid_free_business[covid_free_business['Date'] > covid_free_start_date]

periods_lables = ['Covid Free','Restrictions','Full Lockdown']
periods_data = [covid_free, restrictions, full_lockdown]
for i in range(0, 3):
  print('Number of', periods_lables[i], 'data points:', len(periods_data[i]))


# Data Visualization

In [ ]:
# This section plots the Traffic Downloaded from the ALL cells in a month

def periods_trace_plot_daily(ref_KPI, operation, ylabel):
  # Create a copy of the original dataset
  data_temp = data.copy()

  # Set Daily granularity instead of Hourly granularity
  data_temp['Date'] = pd.to_datetime(data_temp['Date']).dt.date

  # Used to identify days with less measurments
  # pd.set_option('display.max_rows', None)
  # display(data_temp['Date'].value_counts())

  # We drop the data about 31-03-2020 because we are computing daily stats and in this date we have only one measurment for the whole day
  ref = data_temp.set_index('Date').sort_values('Date').drop(datetime.date(year=2020,month=3,day=31)).loc[:, [ref_KPI]]

  print('Original dataset size:', len(ref))
  print('Cleaned dataset size:', len(ref.dropna()))

  # Sum all the "ref_KPI" values with same "Date"
  ref = ref.groupby(level=0).sum()

  # open new figure
  fig, ax = plt.subplots(figsize=(15,8))

  # plot data
  ax.plot(list(range(0,len(ref))), ref[ref_KPI], linestyle='--', lw=2, color='b', alpha=.8) 

  # Set plotting options
  plt.xticks(color='black')
  plt.yticks(color='black')
  plt.grid(1)
  ticks_label = ref.index
  ticks = np.linspace(0, len(ref)-1, 35, dtype=int)
  plt.xticks(ticks = ticks, labels = ticks_label[ticks], fontsize = 14)
  plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left", rotation_mode="anchor") 
  plt.xlabel('Date', color='black', fontsize=14)
  plt.ylabel(ylabel, color='black', fontsize=14) # unit of measure depends on the considered KPI
  plt.title('Median Hourly Trace of '+ref_KPI+' - ALL CELLS', fontsize=14)

  # Draw a red line when there is a period change
  plt.axvline(ticks_label.get_loc(datetime.date(year=2020,month=2,day=16)) ,color = 'r',label = 'Restrictions')
  plt.axvline(ticks_label.get_loc(datetime.date(year=2020,month=3,day=9)) ,color = 'r',label = 'Full Lockdown')

  plt.show()

periods_trace_plot_daily('DL_VOL', 0, 'Bits')
print()
periods_trace_plot_daily('UL_VOL', 0, 'Bits')
print()

In [ ]:
# Cell rappresentation with Geopandas
geo_data = geopandas.GeoDataFrame(locations, geometry=geopandas.points_from_xy(locations.LONG_X, locations.LAT_Y))

# Plot the map
milan = geopandas.read_file(file_path+'ds379_municipi_label.geojson')
ax = milan.plot(color='lightblue', edgecolor='black', figsize=[12,15])

# Plot the cells
geo_data.plot(ax=ax, color='red')
plt.show()


In [ ]:
# Cell differentiation with Geopandas
geo_data_residential = geopandas.GeoDataFrame(residential_locations, geometry=geopandas.points_from_xy(residential_locations.LONG_X, residential_locations.LAT_Y))
geo_data_business = geopandas.GeoDataFrame(business_locations, geometry=geopandas.points_from_xy(business_locations.LONG_X, business_locations.LAT_Y))
geo_data_promiscuous  = geopandas.GeoDataFrame(promiscuous_locations, geometry=geopandas.points_from_xy(promiscuous_locations.LONG_X, promiscuous_locations.LAT_Y))

# Plot the map
milan = geopandas.read_file(file_path+'ds379_municipi_label.geojson')
ax = milan.plot(color='lightblue', edgecolor='black', figsize=[12,15])

# Plot the cells
geo_data_residential.plot(ax=ax, color='green')
geo_data_business.plot(ax=ax, color='blue')
geo_data_promiscuous.plot(ax=ax, color='yellow')
plt.show()

In [ ]:
# This section makes a box plot of the daily statiscs regarding the number of connected 
# users to the cell taken as example. For each day, the following statistics are extracted from the considered
# KPI:
# - Median Value
# - 25th and 75th Quantiles
# - Max and Min values

# For reference about how to read a box plot go here: 
# https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51


def data_dataset_daily(dataset, ref_KPI):
  # Set Daily granularity instead of Hourly granularity
  dataset['Date'] = pd.to_datetime(dataset['Date']).dt.date
  # Set new dataset index to "Date" and keep only the "ref_KPI" column
  return dataset.set_index('Date').sort_values('Date').loc[:, [ref_KPI]]


# If operation == 0 do the sum of the ref_KPI of the day
# If operation == 1 do the average of the ref_KPI of the day
def dataset_operation(dataset, operation):
  if operation == 0:
    dataset = dataset.groupby(level=0).sum()
  else:
    dataset = dataset.groupby(level=0).mean()
  return dataset


def statistical_comparison(periods_lables, median_values, average_values, sd_values, id_1, id_2, ref_KPI):
  print('>>>------', periods_lables[id_1], ref_KPI, 'Variation ------<<<')
  print("Median: difference between [", periods_lables[id_1] ,"] and [",periods_lables[id_2] ,"]: {0:.2f}%".format(((median_values[id_2] - median_values[id_1]) / abs(median_values[id_1])) * 100))
  print("Average: difference between [", periods_lables[id_1] ,"] and [",periods_lables[id_2] ,"]: {0:.2f}%".format(((average_values[id_2] - average_values[id_1]) / abs(average_values[id_1])) * 100))
  print("Std. deviation: difference between [", periods_lables[id_1] ,"] and [",periods_lables[id_2] ,"]: {0:.2f}%".format(((sd_values[id_2] - sd_values[id_1]) / abs(sd_values[id_1])) * 100))


def periods_box_plot_daily(periods, periods_lables, ref_KPI, operation, ylabel):

  periods_copy = []
  periods_data_list = []
  N_periods = len(periods)

  for i in range(N_periods):
    periods_copy.append(periods[i].copy())
    periods_copy[i] = data_dataset_daily(periods_copy[i], ref_KPI)
    periods_copy[i] = periods_copy[i].loc[:, [ref_KPI]].drop(datetime.date(year=2020,month=3,day=31), errors='ignore')
    # Remove null values
    periods_copy[i] = periods_copy[i].dropna()
    periods_copy[i] = dataset_operation(periods_copy[i], ref_KPI)
    # periods_copy have only one KPI but we need "[ref_KPI]" to use the "tolist()" function
    periods_data_list.append(periods_copy[i][ref_KPI].tolist())

  # open new figure
  fig, ax = plt.subplots(figsize=(15,8))

  bplots = []
  for i in range(N_periods):
    bplots.append(ax.boxplot(periods_data_list[i], positions = [i], patch_artist=True))

  for bplot in bplots:
    for patch in bplot['boxes']:
      patch.set_facecolor('lightblue')

  # Set plotting options
  plt.xticks(color='black')
  plt.yticks(color='black')
  plt.grid(1)
  plt.xticks(ticks = list(range(N_periods)), labels = periods_lables, fontsize = 14)
  plt.setp( ax.xaxis.get_majorticklabels(), ha="center") 
  plt.ylabel(ylabel, color='black', fontsize=14)
  if N_periods > 3:
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left", rotation_mode="anchor") 

  plt.title('Box Plot of Median '+ref_KPI+' - ALL CELLs', fontsize=14)
  plt.show()

  print()
  
  median_values = []
  average_values = []
  sd_values = []

  # Print statistical informations:
  for i in range(N_periods):
    print('---------', periods_lables[i], ref_KPI,'---------')
    median_values.append(np.median(periods_data_list[i]))
    print('Median value:', "{0:.2f}".format(median_values[i]))
    average_values.append(np.mean(periods_data_list[i]))
    print('Average value:', "{0:.2f}".format( average_values[i]))
    sd_values.append(np.std(periods_data_list[i]))
    print('Standard deviation:', "{0:.2f}".format(sd_values[i]))

  if N_periods > 3:
    # Statistical comparison between residential and business in the same period
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 0, 1, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 2, 3, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 4, 5, ref_KPI)

    # Statistical comparison between residential periods
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 0, 2, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 0, 4, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 2, 4, ref_KPI)

    # Statistical comparison between business periods
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 1, 3, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 1, 5, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 3, 5, ref_KPI)
  else:
    # Statistical comparison between periods
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 0, 1, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 0, 2, ref_KPI)
    statistical_comparison(periods_lables, median_values, average_values, sd_values, 1, 2, ref_KPI)


In [ ]:
def plot_stats(periods, periods_lables):
  periods_box_plot_daily(periods, periods_lables, 'DL_VOL', 0, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'UL_VOL', 0, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'DL_THR_MAX', 1, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'UL_THR_MAX', 1, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'DL_THR_AVG', 1, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'UL_THR_AVG', 1, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'ERAB_S_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'CS_SR', 1, 'Calls')
  print()
  periods_box_plot_daily(periods, periods_lables, 'RRC_S_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'RRC_RE_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'IntraF_Hout_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'InterF_Hout_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'Hin_SR', 1, 'Success Rate')
  print()
  periods_box_plot_daily(periods, periods_lables, 'PRB_DL_Used_Avg', 1, 'Bits')
  print()
  periods_box_plot_daily(periods, periods_lables, 'PRB_UL_Used_Avg', 1, 'Bits')
  print()

In [ ]:
periods = [covid_free, restrictions, full_lockdown]
plot_stats(periods, periods_lables)

In [ ]:
periods = [covid_free_residential, covid_free_business, restrictions_residential, restrictions_business, full_lockdown_residential, full_lockdown_business]
periods_lables = ['Covid Free Residential', 'Covid Free Business', 'Restrictions Residential', 'Restrictions Business', 'Full Lockdown Residential', 'Full Lockdown Business']
plot_stats(periods, periods_lables)